In [107]:
import requests
from bs4 import BeautifulSoup
import time

# 1. ターゲットURLを組織のリポジトリ一覧ページに変更
pages = 94  # 取得するページ数を指定
repository = []
star = []
language = []

# GitHubへのアクセス時にはUser-Agentを設定することが推奨されます
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

for page in range(1, pages + 1):
    # ページネーションURLを構築
    url = f'https://github.com/orgs/google/repositories?page={page}'

    print(f"\n--- ページ {page} の解析を開始: {url} ---")

    try:
        # GETリクエストを送信 (headersを含める)
        res = requests.get(url, headers=headers)
        res.encoding = res.apparent_encoding
        soup = BeautifulSoup(res.text, 'html.parser')

        # リポジトリのリストを取得
       
        repository_links = soup.select('a.Title-module__anchor--GmXUE.Title-module__inline--oM0P7')
        star_links =  soup.find_all(
            'a', 
            attrs={'aria-label': lambda x: x and 'stars' in x}
        )
        language_links = soup.select('div.ReposListItem-module__Box_9--RH81p')
   

        for i, item in enumerate(repository_links, 1):
            

            # --- リポジトリ名 ---

            repo_name = repository_links[i-1].get_text(strip=True)
            repository.append(repo_name)
            print(f"{repo_name}")
            time.sleep(1)

        for i in range(1, len(star_links) + 1):

            # --- スター数 ---
            # aria-labelに'stars'という文字列が含まれているaタグを探す
            # 例: aria-label="5,000 stars"
            if star_links:
                star_count_label = star_links[i-1]['aria-label'].split(' ')[0]
                star_count_text = star_links[i-1].text.strip()

                star.append(star_count_text)
                print(f"{star_count_text}")
            else:
                print("スター数を表す要素が見つかりませんでした。")

            time.sleep(1)

        for i in range(1, len(language_links) + 1):
            # --- プログラミング言語 ---
            language_name = language_links[i-1].get_text(strip=True) 
            language.append(language_name)
            print(f"{language_name}")
            
            time.sleep(1)
            
            # 結果の表示
        
        print(f"  {i}. リポジトリ: {repo_name} | スター数: {star_count_text} | 言語: {language_name}")
    
    except requests.exceptions.RequestException as e:
        print(f"❌ ページ {page} でリクエストエラーが発生しました: {e}")
        time.sleep(5) 
        continue

    # ページ間の待機時間を確保
    time.sleep(1)

print("-" * 50)
print(f"✅ 完了: 合計 {len(repository)} 件のリポジトリ名、{len(star)} 件のスター数、{len(language)} 件のプログラミング言語を抽出しました。")


--- ページ 1 の解析を開始: https://github.com/orgs/google/repositories?page=1 ---
perfetto
nsjail
nomulus
skia
device-infra
android-cuttlefish
meridian
angle
heir
gn-language-server
zerocopy
matr
docsy
dawn
chromium-policy-vulnfeed
site-kit-wp
langfun
osv-scanner-v1
adk-docs
xls
tunix
quiche
private-compute-libraries
toucan
gson
XNNPACK
kafel
adk-python
mug
biva
5k
3.6k
1.8k
10k
58
571
1.2k
3.8k
608
18
2.1k
9
2.9k
776
7
1.3k
875
2
974
1.4k
1.9k
801
34
48
24k
2.2k
340
16k
444
6
C++
C++
Java
C++
Java
C++
Python
C++
C++
Rust
Rust
Rust
JavaScript
C++
Go
JavaScript
Python
Go
HTML
C++
Python
C++
Kotlin
C++
Java
C
C
Python
Java
C++
  30. リポジトリ: biva | スター数: 6 | 言語: C++

--- ページ 2 の解析を開始: https://github.com/orgs/google/repositories?page=2 ---
dwh-migration-tools
oss-fuzz-vulns
oss-fuzz
ground-android
dive
gvisor
budoux-extension
tabuli
filament
adk-go
Xee
safevalues
init2winit
jetpack-camera-app
orbax
eclipsa-audio-plugin
adk-js
crubit
cel-java
cel-spec
nearby
ink
crosvm
grain
ground-platform
eartheng

In [128]:
import sqlite3
path = ''
db_name = 'google.db'

try:
    # DB接続オプションの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを実行するための言語)を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # SQL分の作成
    # テーブルの作成
    sql = 'CREATE TABLE google (repository TEXT, star TEXT, language TEXT);'

    # SQLの実行
    cur.execute(sql)
except sqlite3.Error as e:
    print(f"エラーが発生しました: {e}")
finally:
    # DBへの接続を閉じる
    conn.close()

In [130]:
path = ''
db_name = 'google.db'

def unwrap_list_recursive(data_list):
    """リスト内の要素を再帰的に処理し、タプル構造を完全に解除して文字列に平坦化する"""
    new_list = []
    for item in data_list:
        current_item = item
        # current_itemがタプルである限り、その中身を取り出し続ける
        while isinstance(current_item, tuple) and len(current_item) > 0:
            current_item = current_item[0]
        
        # 最終的に取り出された文字列または元のNone/N/Aをリストに追加
        new_list.append(current_item)
    return new_list

new_repository = unwrap_list_recursive(repository)
new_star = unwrap_list_recursive(star)
new_language = unwrap_list_recursive(language)

try:    
    # DB接続オプションの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを実行するための言語)を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # SQL分の作成
    # 複数データの挿入
    # INSERT INTO テーブル名　VALUES (列に対応したプレースホルダーをカンマ区切りで);
    # 文字列データを挿入する場合はシングルクォート(')で囲む必要がある
    sql = "INSERT INTO google (repository, star, language) VALUES (?, ?, ?);"

    # 挿入するデータの用意
    max_len = max(len(new_repository), len(new_star), len(new_language))
    repository_data = new_repository + [None] * (max_len - len(new_repository))
    star_data = new_star + [None] * (max_len - len(new_star))
    language_data = new_language + [None] * (max_len - len(new_language))


    # 2. 3つのリストの要素を組み合わせて、行ごとのタプルにする (zipを使用)
    data_to_insert = []
    for repo, star_count, lang in zip(repository_data, star_data, language_data):
        # プレースホルダーの数 (3つ) に完全に一致するタプルを作成
        data_to_insert.append((repo, star_count, lang))
    # SQLの実行
    cur.executemany(sql, data_to_insert)

    # コミット（DBに変更を反映）
    conn.commit()

except sqlite3.Error as e:
    print(f"エラーが発生しました: {e}")

finally:
    # DBへの接続を閉じる
    conn.close()

    
